In [ ]:
import sys
!{sys.executable} -m pip install dask[complete]

In [2]:
import dask
import dask.bag as db
import os

# Define the directory containing the text files
directory = '/mnt/storage/Kilser_Sentinal/data_formated_jainil4/labels/'

# Get a list of all txt files in the directory
txt_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.txt')]

print(len(txt_files))

# Function to read a single file and return its contents along with the filename
def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read().replace('\n', ' ')  # Read the file and replace newlines with spaces
    filename = os.path.basename(file_path)
    return {'filename': filename, 'content': content}

# Create a Dask bag from the list of files
dask_bag = db.from_sequence(txt_files).map(read_file)

# Convert the Dask bag to a Dask dataframe
dask_df = dask_bag.to_dataframe()

# Persist the dataframe (optional, to optimize computations)
dask_df = dask_df.persist()
print("completed dataframe")

# If you want to save as Parquet instead of CSV
# dask_df.to_parquet('output_directory/processed_files.parquet', write_index=False)


131465
completed dataframe


In [3]:
split_df = dask_df['content'].str.split(expand=True, n=4)
max_cols = len(split_df.columns)

# Rename the columns for clarity
split_df = split_df.rename(columns={i: f'col_{i+1}' for i in range(max_cols)})

# Combine the original 'filename' column with the split columns
result_df = dask.dataframe.concat([dask_df[['filename']], split_df], axis=1)

# Persist the dataframe (optional, to optimize computations)
result_df = result_df.persist()

/home/japmyy/temp/lib/python3.6/site-packages/dask/dataframe/multi.py:1193: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  "Concatenating dataframes with unknown divisions.\n"


In [4]:

result_df.to_csv('/mnt/storage/Kilser_Sentinal/labels_processed.csv', single_file=True)

['/mnt/storage/Kilser_Sentinal/labels_processed.csv']

In [5]:
import pandas as pd
df = pd.read_csv("/mnt/storage/Kilser_Sentinal/labels_processed.csv")

In [6]:
df.head()

,Unnamed: 0,filename,col_1,col_2,col_3,col_4,col_5
0,0,video_6f892664-761b-4f80-af7f-19efd72f8628_fra...,NaN,NaN,NaN,NaN,NaN
1,1,video_5f87e18e-ffdf-4957-9899-b5414cda66e2_fra...,0.0,0.701600,0.499934,0.043527,0.063624
2,2,video_862d3cfb-83e8-4d4b-b7bc-aad791bd8158_fra...,NaN,NaN,NaN,NaN,NaN
3,3,video_fd55126b-c832-433d-911e-e548e3bfae30_fra...,1.0,0.505000,0.528750,0.117500,0.070833
4,4,video_59148601-615f-4b39-b028-5b9ad5ce29ba_fra...,2.0,0.920703,0.624074,0.157031,0.371296


In [9]:
df["col_1"].value_counts(dropna=False)

NaN    97457
1.0    12496
2.0    11229
0.0    10283
Name: col_1, dtype: int64